In [5]:
##Experiment on concatenating raw expressions with embedding vectors

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from scipy.stats import norm
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import matplotlib.pyplot as plt


In [6]:
from sklearn.metrics import precision_score, recall_score, balanced_accuracy_score
def calc_results_simple(X, y, train_index, test_index, clf):
    X, y = X.to_numpy(), y.to_numpy(dtype=np.int64)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf.fit(X_train, y_train)
    y_pred  = clf.predict(X_test)
    y_pred_prob = clf.predict_proba(X_test)[:,1]
    acc = balanced_accuracy_score(y_test, y_pred)

    recall_0 =  recall_score(y_test, y_pred, pos_label=0)
    recall_1 =  recall_score(y_test, y_pred, pos_label=1)
    prec_0 = precision_score(y_test, y_pred, pos_label=0)
    prec_1 = precision_score(y_test, y_pred, pos_label=1)
    auc = roc_auc_score(y_test, y_pred_prob)

    return np.array([[acc, recall_0, prec_0, recall_1, prec_1 ,auc]])

#cross_validation
def run_cross_val(X, y, params, n_folds=5, random_seed=42):
    res = np.empty(shape=[0, 6])
    clf = XGBClassifier(**params, n_jobs=8)
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        res = np.append(res, calc_results_simple(X, y, train_index, test_index, clf), axis=0)
    return res, clf

def print_score_comparison(raw_score, emb_score, target_feature="RFS",
                           header_1="Raw Score", header_2="Embedding Score"):
    print("\t\t{0}\n\t\t\t{1}\t\t{2}".format(target_feature, header_1, header_2))
    print("\t\t-----------------------------------------------")
    print("balanced_accuracy:\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["balanced_accuracy"].mean(), emb_score["balanced_accuracy"].mean()))
    print("precision_0:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["precision_0"].mean(), emb_score["precision_0"].mean()))
    print("recall_0:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["recall_0"].mean(), emb_score["recall_0"].mean()))
    print("precision_1:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["precision_1"].mean(), emb_score["precision_1"].mean()))
    print("recall_1:\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["recall_1"].mean(), emb_score["recall_1"].mean()))
    print("auc:\t\t\t{0:.3%}\t\t\t{1:.3%}\n".format(raw_score["auc"].mean(), emb_score["auc"].mean()))

def find_misclassified_patients(df, clf, X, y):
    y_test = y.to_numpy()
    X_test = X.to_numpy()
    miss = np.where(y_test != clf.predict(X_test))
    return df.iloc[miss]["patient_ID"].to_numpy(dtype=np.int64)

def calc_overlap(a, b):
    intr = np.intersect1d(a, b)
    union = np.union1d(a, b)
    return intr, (len(intr) / len(union))

def print_overlap(model1, model2, intr, perc):
    print("{0} patients misclassified by {1} and {2} - {3:.1%} overlap\n".format(len(intr) ,model1, model2, perc))

def write_misclassified(file_name, ls):
    with open("datasets/" + file_name + ".txt", "w") as f:
        for p in ls:
            f.write(str(p) + "\n")


In [7]:
params = {'n_estimators': [300, 400, 500, 600, 700],
              'learning_rate': [0.01, 0.02, 0.03, 0.05, 0.07],
              'gamma': [0.5, 1, 1.5, 2, 5],
              'max_depth': [3, 4, 5, 6],
              'subsample': [0.6, 0.8, 1.0],
              'colsample_bytree': [0.6, 0.8, 1.0],
              'min_child_weight': [1, 2, 3, 4, 5]}
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time

    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

def param_tuning(X, y, n_folds=5, param_comb=25, scoring='roc_auc', jobs=12):
    xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    rand_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring=scoring, n_jobs=jobs,
                                   cv=skf.split(X, y), verbose=3, random_state=42)

    start_time = timer(None) # timing starts from this point for "start_time" variable
    rand_search.fit(X, y)
    timer(start_time)
    print("Best Score: {:.3%}".format(rand_search.best_score_))
    print(rand_search.best_params_)
    return rand_search

In [8]:
ge_df = pd.read_csv("datasets/merged-combat15.csv")
outcome_df = pd.read_csv("datasets/combat15outcomes.csv")
pos_outcome_df = outcome_df[["patient_ID", "posOutcome"]].dropna(axis=0, subset=["posOutcome"])
pos_outcome_df.posOutcome = pos_outcome_df.posOutcome.astype(int)
ge_outcome_df = pd.merge(pos_outcome_df, ge_df, on="patient_ID")

In [11]:
cols = ['patient_ID', 'posOutcome', 'CDX4','GLRA1', 'OR12D3', 'DSCR4', 'HOXB8', 'C9', 'MTNR1B', 'MOS', 'HSD17B3', 'FGF20', 'KCNH4', 'ATP4B', 'CPB2', 'CRYBB1', 'ANGPTL3', 'MYH8', 'GYS2', 'SLC25A21', 'TAS2R7', 'F11', 'GABRA6', 'MYT1L', 'DEFB126', 'RPL18', 'GABRQ', 'ZFP37', 'PIP5K1B', 'MCM5', 'PRKAA1', 'WDR76', 'CHRM4', 'RPS6KC1', 'EIF1AY', 'WNT1', 'SCN3B', 'NLGN4Y', 'MAGEB1', 'NUDC', 'HIGD1A', 'OXCT2', 'GALR2', 'EEF1B2', 'RXRG', 'CALCA', 'TEX13A', 'CST3', 'IGFBP4', 'CRYGA', 'ESR1', 'ZNF750']

tmp_df = ge_outcome_df[cols]

xgb50_emb_df = pd.read_csv("datasets/embedding-vectors/property_vector_xgb50_withoutplnresult_2020-12-23.csv", sep="\t")
xgb50_outcome_df = pd.merge(tmp_df, xgb50_emb_df, on="patient_ID")
X_xgb50_outcome, y_xg50_outcome = xgb50_outcome_df[xgb50_outcome_df.columns.difference(["patient_ID", "posOutcome"])], xgb50_outcome_df["posOutcome"]
xgb50_outcome_df.head()

,patient_ID,posOutcome,CDX4,GLRA1,OR12D3,DSCR4,HOXB8,C9,MTNR1B,MOS,...,2221,2222,2223,2224,2225,2226,2227,2228,2229,2230
0,22449,0,4.393932,4.756301,3.668209,3.813140,3.149279,4.091114,3.778200,4.149525,...,2.983482e-06,-5.667081e-06,-2.578453e-06,3.233320e-06,3.162931e-06,-1.622541e-07,-2.522139e-06,7.292363e-07,2.538494e-07,2.798644e-08
1,22450,0,3.735445,3.453197,3.008127,2.500197,3.025658,3.265710,2.909130,2.990024,...,2.859682e-06,7.411624e-07,-3.327798e-07,-7.720635e-07,-3.062798e-06,-2.325743e-07,-6.893824e-07,-1.527674e-06,2.051379e-06,3.103411e-07
2,22451,0,3.504602,3.591334,3.487448,2.710443,2.786988,3.904477,2.879539,3.585594,...,1.504380e-06,6.565640e-06,4.819232e-07,-2.377132e-06,7.905603e-07,-1.038275e-06,-9.005510e-07,9.035374e-07,4.423281e-07,5.704054e-08
3,22452,0,2.862134,3.326514,3.346279,3.676626,4.426359,3.111246,3.447916,3.153298,...,6.847706e-07,2.294943e-06,9.302902e-07,-7.566930e-07,-5.137400e-07,2.965493e-06,1.247254e-06,2.925552e-07,9.689832e-07,5.092384e-09
4,22453,1,3.706718,4.106301,3.579494,3.123646,3.254895,3.480252,3.673946,3.867726,...,-5.978619e-07,6.835914e-08,-1.627280e-06,-2.343308e-06,-3.645692e-07,-2.182235e-06,1.108295e-06,5.456489e-07,2.321873e-06,-1.789428e-07


In [12]:
rand_search_xg50 = param_tuning(X_xgb50_outcome, y_xg50_outcome, jobs=14)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[11:38:44] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[11:38:44] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 44 minutes and 31.42 seconds.
Best Score: 77.879%
{'subsample': 0.8, 'n_estimators': 600, 'min_child_weight': 3, 'max_depth': 6, 'learning_rate': 0.01, 'gamma': 5, 'colsample_bytree': 0.6}


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   4 tasks      | elapsed:  2.9min
[Parallel(n_jobs=14)]: Done 125 out of 125 | elapsed: 40.5min finished


In [15]:
outcome_xg50_params = {'subsample': 0.8,
 'n_estimators': 600,
 'min_child_weight': 3,
 'max_depth': 6,
 'learning_rate': 0.01,
 'gamma': 5,
 'colsample_bytree': 0.6}

In [16]:
outcome_xg50_scores, clf_xg50 = run_cross_val(X_xgb50_outcome, y_xg50_outcome, outcome_xg50_params)
outcome_xg50_df = pd.DataFrame(data=outcome_xg50_scores, columns=["balanced_accuracy", "recall_0", "precision_0", "recall_1", "precision_1", "auc"])
outcome_xg50_df.describe()

[12:06:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:07:26] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:07:54] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:08:23] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

,balanced_accuracy,recall_0,precision_0,recall_1,precision_1,auc
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.699896,0.516319,0.726036,0.883472,0.752908,0.778792
std,0.039918,0.069596,0.054114,0.022830,0.028664,0.025239
min,0.655081,0.427711,0.655172,0.855072,0.721713,0.741343
25%,0.659704,0.455090,0.702970,0.876812,0.722222,0.764364
50%,0.715514,0.554217,0.730159,0.876812,0.765823,0.792452
75%,0.727562,0.566265,0.738462,0.891697,0.775641,0.794681
max,0.741616,0.578313,0.803419,0.916968,0.779141,0.801118


In [17]:
moses_cols = ["patient_ID", "posOutcome", "PRND", "FRS3", "FCN3", "DSCR4", "BRCA2", "CXCL6", "LMX1B", "DLX5", "OMP", "ADH6", "PGAP1", "ART3", "BCHE", "FGB", "IL1RAPL1", "FSTL4", "ASGR1", "ZNF135", "DLL3", "NPHS2", "ANGPT2", "GLP2R", "GRIA3", "HOXB8", "MSC", "PLA2R1", "CYP2F1", "TAS2R7", "NKX6-1", "WNT11", "CHST11", "CLCA4", "ENPEP", "PAH", "WFDC1", "CHGA", "SEZ6L", "UGT2A3", "PRDM16", "GALR2", "GUCA1A", "CASQ1", "NOS1AP", "CACNA2D3", "FHOD3", "SRGAP3", "TMOD2", "ATOH1", "SLC6A1", "HAS1"]

tmp_df = ge_outcome_df[moses_cols]
emb_df = pd.read_csv("datasets/embedding-vectors/property_vector_moses50_withoutplnresult_2020-12-23.csv", sep="\t")
emb_outcome_df = pd.merge(tmp_df, emb_df, on="patient_ID")
X_emb_outcome, y_emb_outcome = emb_outcome_df[emb_outcome_df.columns.difference(["patient_ID", "posOutcome"])], emb_outcome_df["posOutcome"]
emb_outcome_df.head()

,patient_ID,posOutcome,PRND,FRS3,FCN3,DSCR4,BRCA2,CXCL6,LMX1B,DLX5,...,2223,2224,2225,2226,2227,2228,2229,2230,2231,2232
0,22449,0,4.773548,3.792942,3.591425,3.813140,3.842011,3.301166,3.155381,4.092754,...,3.837437e-06,3.315117e-06,8.110773e-07,5.140755e-07,-0.000002,4.659053e-07,1.246811e-07,-5.459261e-07,-1.940580e-08,1.044367e-06
1,22450,0,4.050956,3.596728,3.603971,2.500197,2.761469,3.679678,3.406322,3.698481,...,5.243294e-06,-1.819026e-06,1.511906e-06,-5.381149e-08,0.000001,1.001947e-06,-2.010190e-06,-2.846180e-07,5.900286e-07,2.206977e-07
2,22451,0,5.213503,3.892048,3.655383,2.710443,2.561722,3.748453,3.964545,4.125640,...,5.728294e-06,7.688957e-09,-6.358236e-08,-1.185732e-06,0.000002,1.467577e-06,1.072813e-06,3.649615e-07,5.383282e-07,6.781582e-07
3,22452,0,3.443242,3.713757,3.370449,3.676626,3.947755,2.890541,2.987402,3.919090,...,-4.844169e-06,2.724217e-06,3.021055e-06,-1.983205e-06,0.000001,4.689168e-06,3.073779e-06,-1.868303e-07,2.015108e-08,-2.133324e-07
4,22453,1,4.237601,3.800724,3.259677,3.123646,3.354961,3.029855,3.116395,3.882619,...,6.608744e-07,3.468912e-06,3.783993e-06,-2.091969e-06,0.000002,1.407070e-06,2.260274e-06,7.934927e-07,-3.723606e-07,-4.706951e-07


In [18]:
rand_search_moses50 = param_tuning(X_emb_outcome, y_emb_outcome, jobs=14)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[12:54:17] WARNING: ../src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:54:18] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Time taken: 0 hours 42 minutes and 46.71 seconds.
Best Score: 76.576%
{'subsample': 1.0, 'n_estimators': 400, 'min_child_weight': 3, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 1, 'colsample_bytree': 0.6}


[Parallel(n_jobs=14)]: Using backend LokyBackend with 14 concurrent workers.
[Parallel(n_jobs=14)]: Done   4 tasks      | elapsed:  2.9min
[Parallel(n_jobs=14)]: Done 125 out of 125 | elapsed: 40.1min finished


In [21]:
outcome_moses50_params = {'subsample': 1.0,
 'n_estimators': 400,
 'min_child_weight': 3,
 'max_depth': 5,
 'learning_rate': 0.01,
 'gamma': 1,
 'colsample_bytree': 0.6}

In [22]:
outcome_moses50_scores, clf_moses50 = run_cross_val(X_emb_outcome, y_emb_outcome, outcome_moses50_params)
outcome_moses50_df = pd.DataFrame(data=outcome_moses50_scores, columns=["balanced_accuracy", "recall_0", "precision_0", "recall_1", "precision_1", "auc"])
outcome_moses50_df.describe()

[13:52:57] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:53:16] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:53:35] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:53:55] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

,balanced_accuracy,recall_0,precision_0,recall_1,precision_1,auc
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.691251,0.487432,0.735507,0.895069,0.744087,0.765764
std,0.033616,0.052509,0.053586,0.019852,0.022706,0.019368
min,0.655496,0.431138,0.679245,0.877256,0.720588,0.742710
25%,0.659409,0.433735,0.699029,0.884058,0.721068,0.747031
50%,0.699275,0.500000,0.733333,0.887681,0.749245,0.774860
75%,0.707089,0.530120,0.747748,0.898551,0.757764,0.781393
max,0.734983,0.542169,0.818182,0.927798,0.771772,0.782827


In [24]:
print_score_comparison(outcome_moses50_df, outcome_xg50_df, target_feature="posOutcome", header_1="MOSES 50", header_2="Xgboost 50")

		posOutcome
			MOSES 50		Xgboost 50
		-----------------------------------------------
balanced_accuracy:	69.125%			69.990%

precision_0:		73.551%			72.604%

recall_0:		48.743%			51.632%

precision_1:		74.409%			75.291%

recall_1:		89.507%			88.347%

auc:			76.576%			77.879%

